In [1]:
import numpy as np
import pandas as pd
from pymongo import MongoClient
import string
import re
import nltk
from nltk.tokenize import TweetTokenizer
import pprint
from pandas.io.json import json_normalize
import unicodedata
from pandas import ExcelWriter #para exportar a excel


In [2]:
# Acceder a las colecciones
MONGO_HOST= 'mongodb://localhost/tweetsdb'
client = MongoClient(MONGO_HOST)
db = client.tweetsdb
col = db.col_query1_spanish_stream

In [3]:
# Crear un dataframe con los datos
cursor1 = col.find()
dataframe = list(cursor1)
df = json_normalize(dataframe)

In [4]:
list(df.columns.values)

['_id',
 'contributors',
 'coordinates',
 'coordinates.coordinates',
 'coordinates.type',
 'created_at',
 'display_text_range',
 'entities.hashtags',
 'entities.media',
 'entities.symbols',
 'entities.urls',
 'entities.user_mentions',
 'extended_entities.media',
 'extended_tweet.display_text_range',
 'extended_tweet.entities.hashtags',
 'extended_tweet.entities.media',
 'extended_tweet.entities.symbols',
 'extended_tweet.entities.urls',
 'extended_tweet.entities.user_mentions',
 'extended_tweet.extended_entities.media',
 'extended_tweet.full_text',
 'favorite_count',
 'favorited',
 'filter_level',
 'geo',
 'geo.coordinates',
 'geo.type',
 'id',
 'id_str',
 'in_reply_to_screen_name',
 'in_reply_to_status_id',
 'in_reply_to_status_id_str',
 'in_reply_to_user_id',
 'in_reply_to_user_id_str',
 'is_quote_status',
 'lang',
 'place',
 'place.bounding_box.coordinates',
 'place.bounding_box.type',
 'place.country',
 'place.country_code',
 'place.full_name',
 'place.id',
 'place.name',
 'place.p

In [5]:
# Me quedo solo con algunos campos
df1=df.drop(df.columns.difference(['_id','lang','source','user.description','user.id','user.lang','user.name','user.screen_name','user.url','text']),1)

In [7]:
# Convierte texto a minúsculas, tokeniza, quita acentos y "ñ"
def remove_accents(input_str):
    try:
        nfkd_form = unicodedata.normalize('NFKD', str(input_str))
        only_ascii = nfkd_form.encode('ascii', errors='ignore').decode('utf-8').replace(u'\xf1', 'n')
        text = only_ascii.lower()
    except NameError:
        pass
        
    return text

In [12]:
# Convierto el texto del campo user.name 
df1.loc[:, 'user.name'] = df1['user.name'].apply(remove_accents)

In [13]:
# Lista con nombres y apellidos de personas en español e ingles
nombres = [line.strip() for line in open('nombres.txt', 'r')]

In [14]:
pat = '|'.join([r'\b{}\b'.format(x) for x in nombres]) # patrón para chequear en el user.name las coincidencias con los nombres de personas

In [15]:
# Cuenta el total de palabras coincidentes con los nombres incluidos en la lista nombres 
def word_count(text):
    word_count = 0
    for word in text.split():
        if word in nombres:
            word_count += 1
        else:
            word_count
    return word_count

In [16]:
# % de palabras incluidas en user.name que coinciden con la lista de nombres y apellidos en español e ingles
df1['Check_user name'] = (df1['user.name'].apply(word_count))*100/(df1['user.name'].str.split().str.len())

In [17]:
# Convierto el texto del campo user.description 
df1.loc[:, 'user.description'] = df1['user.description'].apply(remove_accents)

In [18]:
# Lista de palabras que puede aparecer en la descripción si se trata de una persona
persona = ['emprendedor','persona', 'licenciado', 'ingeniero','freelance','i']

In [19]:
# Lista de palabras que no deberían aparecer en la descripción si se trata de una persona
no_persona = [line.strip() for line in open('term_emp.txt', 'r')]

In [20]:
# patrón de búsqueda de las palabras incluidas en la lista persona y en la de no_persona
patron_2 = '|'.join([r'\b{}\b'.format(x) for x in persona]) 
patron_3 = '|'.join([r'\b{}\b'.format(x) for x in no_persona]) 

In [21]:
# Incluir una nueva columna con valor 1 si la columna user.description contiene alguna de las palabras en la lista persona
df1["Check_descrip_1"] = (df1['user.description'].str.contains(patron_2).astype(int)|(~df1['user.description'].str.contains(patron_3))|(~df1['user.name'].str.contains(patron_3))).astype(int)

In [22]:
# Variable con la condicion para asignar cada tweet a una persona o a una empresa

cond= (df1["Check_descrip_1"] == 1) & (df1['Check_user name'] >= 30)

In [23]:
# Asignar a la columna es_persona 1 con la condicion anterior
df1['Es_persona']=cond.astype(int)

In [25]:
df1[:20]

,_id,lang,source,text,user.description,user.id,user.lang,user.name,user.screen_name,user.url,Source_name,Check_source,Check_url,Check_user name,Check_descrip_1,Es_persona
0,5a11f43d3842ed1f787d8d82,es,"<a href=""http://trailerteam.tumblr.com"" rel=""n...",RT @hugomaravilla: No #BigData No #equity for...,"on a kraftwerk noise: ""i am a raa-bot..""\nthis...",928530080955011072,en,kohoraabot,Kohoraa_bot,None,Kohoraa_bot,0,1,0.0,1,0
1,5a11f4f93842ed1f787d8d83,es,"<a href=""http://twitter.com/download/android"" ...",RT @miguelon_nebot: “En los dos últimos años s...,"libertad individual, propiedad privada, autode...",4756586735,es,libertariansk,libertariansk,http://vocesdelibertad.com,Twitter for Android,1,0,0.0,1,0
2,5a11f5163842ed1f787d8d84,es,"<a href=""http://twitter.com/download/iphone"" r...",RT @nicestream: Nuestro @diegobarcia86 acompañ...,none,75660350,es,gbarcia,gerardobarcia,None,Twitter for iPhone,1,1,0.0,1,0
3,5a11f70d3842ed1f787d8d85,es,"<a href=""http://twitter.com/download/android"" ...",RT @chemaalonso: El lado del mal - Cómo hacer ...,tecnico superior en administracion de sistemas...,303449627,es,alvaro solas lara,AlvaroSolas,None,Twitter for Android,1,1,100.0,1,1
4,5a11f8e13842ed1f787d8d86,es,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",Cuatro ejemplos de cómo el #bigdata cambia el ...,emprendedor abierto a retos. esade. mba execut...,320339238,es,manuel bonilla,manuelbonilla94,http://www.suma.es,Twitter Web Client,0,0,100.0,1,1
5,5a11f9343842ed1f787d8d87,es,"<a href=""http://twitter.com/download/iphone"" r...",RT @chemaalonso: El lado del mal - Cómo hacer ...,la curiosidad mato al gato... pero el gato mur...,551317364,es,alvaro cerda,AlvaroOficial_3,None,Twitter for iPhone,1,1,100.0,1,1
6,5a11f9913842ed1f787d8d88,es,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",Variable Data Types for Excel #MSExcel #BigDat...,"#msexcel mvp, author, trainer, and project dev...",270052245,en,tom urtis/atlas,TomUrtis,http://www.atlaspm.com/,Twitter Web Client,0,0,50.0,1,1
7,5a11f9d83842ed1f787d8d89,es,"<a href=""https://ifttt.com"" rel=""nofollow"">IFT...",RT:TomUrtis: Variable Data Types for Excel #MS...,father ov 3 who supports lfc & likes 2 do odd ...,230403822,en,andy sugden,AndySugs,None,IFTTT,0,1,100.0,1,1
8,5a11fa093842ed1f787d8d8a,es,"<a href=""http://twitter.com/download/iphone"" r...",RT @rooter_es: Estuvimos en el #outthink17 con...,"director general de @rooter_es, experto intern...",121178359,es,conrado castillo,ConradoCastillo,http://conradocastillo-serna.blogspot.com.es/p...,Twitter for iPhone,1,0,100.0,1,1
9,5a11fa393842ed1f787d8d8b,es,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",RT @AndySugs: RT:TomUrtis: Variable Data Types...,@intellisystem_ & http://www.randieri.com #fo...,1940230142,it,cristian randieri,C_Randieri,http://www.intellisystem.it,Twitter Web Client,0,0,50.0,1,1


In [26]:
#df1.applymap(type) # comprobar tipos

In [27]:
df1['_id'] = df1['_id'].astype(str) # cambiar el tipo de la columna para poder exportar a excel

In [28]:
#df1.applymap(type) # comprobar tipos

In [30]:
writer = pd.ExcelWriter('/Users/Silvia/Filtro_pers.xlsx', engine='xlsxwriter')
df1.to_excel(writer,'Sheet1')
writer.save()